## Random Forest with 3-fold CV, StandardScaler and optuna on dataset3.0
dataset: 11 features,  subsampled dataset

20220926

In [2]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import optuna as opt
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.model_selection import cross_val_score

In [3]:
#import data
train_df = pd.read_parquet("train_main.parquet.snappy")
test_df = pd.read_parquet("test_main.parquet.snappy")

In [18]:
X_train = train_df.drop(['r_id','r_useful'],axis=1)
y_train = train_df['r_useful']
X_test = test_df.drop(['r_id','r_useful'],axis=1)
y_test = test_df['r_useful']

# standardise the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [4]:
def objective(trial):
    start = time.time()
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 1, 500, 50),
        "max_depth": trial.suggest_int('max_depth', 1, 500, 50),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "max_features" : trial.suggest_categorical("max_features", ['sqrt', 'log2']),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20), 
        "random_state": 760, 
        "n_jobs": -1
    }
    print("Currently running with:")
    print(params)
    
    model = RandomForestRegressor(**params)
    
    # model.fit(X_train, y_train)
    cvscore = np.sqrt(-np.mean(cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv=3)))
    # y_pred = model.predict(X_val)
    print("Time cost:", time.time()-start)
    # return mean_squared_error(y_val, y_pred, squared=False)
    return cvscore

In [5]:
study = opt.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)
study.best_params

[I 2022-09-27 03:37:50,258] A new study created in memory with name: no-name-cb859c65-3fd3-44f6-a15e-bd0cd41c6c10


Currently running with:
{'n_estimators': 51, 'max_depth': 201, 'min_samples_split': 10, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-27 03:38:02,088] Trial 0 finished with value: 3.607386181203533 and parameters: {'n_estimators': 51, 'max_depth': 201, 'min_samples_split': 10, 'max_features': 'sqrt', 'min_samples_leaf': 10}. Best is trial 0 with value: 3.607386181203533.


Time cost: 11.827013969421387
Currently running with:
{'n_estimators': 251, 'max_depth': 51, 'min_samples_split': 12, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-27 03:39:22,836] Trial 1 finished with value: 3.5871712605659782 and parameters: {'n_estimators': 251, 'max_depth': 51, 'min_samples_split': 12, 'max_features': 'sqrt', 'min_samples_leaf': 8}. Best is trial 1 with value: 3.5871712605659782.


...

[I 2022-09-27 04:35:30,930] Trial 48 finished with value: 3.5566728884756427 and parameters: {'n_estimators': 251, 'max_depth': 101, 'min_samples_split': 6, 'max_features': 'sqrt', 'min_samples_leaf': 3}. Best is trial 42 with value: 3.5474646810677153.


Time cost: 76.34194278717041
Currently running with:
{'n_estimators': 201, 'max_depth': 51, 'min_samples_split': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-27 04:36:44,521] Trial 49 finished with value: 3.549025828527312 and parameters: {'n_estimators': 201, 'max_depth': 51, 'min_samples_split': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1}. Best is trial 42 with value: 3.5474646810677153.


Time cost: 73.58997440338135


{'n_estimators': 301,
 'max_depth': 101,
 'min_samples_split': 7,
 'max_features': 'log2',
 'min_samples_leaf': 1}

In [5]:
#optimal model
# 'n_estimators': 301, 'max_depth': 101, 'min_samples_split': 7, 'max_features': 'log2', 'min_samples_leaf': 1
omodel = RandomForestRegressor(n_estimators = 301, 
                               max_depth = 101, 
                               min_samples_leaf= 1,
                               min_samples_split= 7,
                               max_features= 'log2', random_state = 760, n_jobs=-1)


omodel.fit(X_train, y_train)

In [6]:
#predict train set results
y_pred = omodel.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
mae = mean_absolute_error(y_train, y_pred) 
print(f"\n model train score -> RMSE:", rmse, "MAE:", mae)


#predict test set results
y_pred_test = omodel.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
mae = mean_absolute_error(y_test, y_pred_test) 
print(f"\n model test score -> RMSE:", rmse, "MAE:", mae)


 model train score -> RMSE: 2.0704855821728825 MAE: 0.8826368503630423

 model test score -> RMSE: 4.102792258442667 MAE: 1.6248397899885325
